In [106]:
import scipy.special as sps
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from itertools import combinations, permutations
from numpy.linalg import inv
from collections import defaultdict
import math
import time
import keras

from keras.utils import np_utils
import keras.callbacks as cb
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import to_categorical

np.seterr(all='print')

{'divide': 'print', 'invalid': 'print', 'over': 'print', 'under': 'print'}

In [107]:
dummy_feature_List = ['country_group_CAN','country_group_EURO', 'country_group_USA', 'Position_C',
       'Position_D', 'Position_L', 'Position_R'] 
def standardize(s_df):
    for col in s_df.columns.values:
        col_mean = s_df[col].mean()
        col_std = s_df[col].std()

        #         mean of interaction terms of two discrete variable are zero. those columns are
        #         filtered while standizing as it cause singular matrix
        if (col_mean != 0 or col_std != 0) or col not in dummy_feature_List :
            s_df[col] = s_df[col].apply(lambda x: (x - col_mean) / float(col_std))

    return s_df


def preprocessing(x_df):
    x_df = pd.get_dummies(x_df, prefix=['country_group', 'Position'], columns=['country_group', 'Position'])
    x_df = x_df.apply(pd.to_numeric, args=('coerce',))

#     # # adding interaction terms
#     for col_1, col_2 in combinations(x_df.columns, 2):
#         cond1 = col_1 not in dummy_feature_List
#         cond2 = col_2 not in dummy_feature_List
#         if cond1 or cond2:
#             x_df['{}*{}'.format(col_1, col_2)] = np.multiply(x_df[col_1], x_df[col_2])
#     # x_df.to_csv('ex.csv', sep='\t')

# #     standardization
#     x_df = standardize(x_df)
# #     y_df = standardize(y_df)

    #     deleteing column = 0
    x_df = x_df.loc[:, (x_df != 0).any(axis=0)]
#     x_df.insert(loc=0, column='x0', value=1)

    return x_df

def calculate_negative_log_likelihood(target, weights, features):
    
    z = np.dot(weights,features)
    y = sps.expit(z)
    mask = np.isinf(y)
    y[mask] = -z[mask]
    
    likelihood = np.mean(np.sum((1-target)*z + np.log(y)))
    
    return likelihood


def sigmoid(x):
    
    return 1 / (1 + np.exp(-x))


In [108]:
def load_dataset():
    input = os.path.join("../MLAssign2/Least-Square-Regression/",'Model_Trees_Full_Dataset', 'preprocessed_datasets.csv')
    data = pd.read_csv(input)

    # random shuffle
    data = data.iloc[np.random.permutation(len(data))]
    # data[u'GP_greater_than_0'] = data[u'GP_greater_than_0'].map({'yes': 1, 'no': 0})

    training_df = data[data[u'DraftYear'].isin([2004, 2005, 2006])]
    testing_df = data[data[u'DraftYear'] == 2007]

    drop_class = [u'id', u'Country', u'Overall', u'PlayerName', u'sum_7yr_TOI', u'DraftYear',"GP_greater_than_0"]
    training_df.drop(drop_class, inplace=True, axis=1)
    testing_df.drop(drop_class, inplace=True, axis=1)

    y_train_df = training_df.filter([u'sum_7yr_GP'])
    x_train_df = training_df.drop([u'sum_7yr_GP'], axis=1)

    y_test_df = testing_df.filter([u'sum_7yr_GP'])
    x_test_df = testing_df.drop([u'sum_7yr_GP'], axis=1)

    x_train_df_processed = preprocessing(x_train_df)
    x_test_df_processed = preprocessing(x_test_df)

    y_train = y_train_df.values
    x_train = x_train_df_processed.values
    y_test = y_test_df.values
    x_test = x_test_df_processed.values
    
    return (x_train,y_train), (x_test, y_test)

In [109]:
tol = 0.00001

# Step size for gradient descent.
etas = [0.5, 0.3, 0.1, 0.05, 0.01]
# etas = [0.1, 0.05, 0.01]

(x_train,y_train), (x_test, y_test) = load_dataset()

/home/aarthy/anaconda3/envs/NLP/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/aarthy/anaconda3/envs/NLP/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [110]:
x_train

array([[ 18,  74, 170, ...,   1,   0,   0],
       [ 19,  72, 194, ...,   1,   0,   0],
       [ 18,  73, 183, ...,   0,   0,   0],
       ..., 
       [ 19,  80, 249, ...,   0,   1,   0],
       [ 18,  72, 203, ...,   0,   0,   0],
       [ 20,  74, 220, ...,   1,   0,   0]])

In [111]:
print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

(637, 22)
(637, 1)
(191, 22)
(191, 1)


In [119]:
start_time = time.time()
model = Sequential()
model.add(Dense(units=50, input_dim=22, activation='relu' ))
model.add(Dense(units=1000, activation='relu' ))
model.add(Dense(1,activation='linear',use_bias=True))
# model.compile(loss="mean_squared_error", optimizer ='adam', metrics=['accuracy'])

# rms = RMSprop()
sgd = keras.optimizers.SGD(lr=1)
model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['accuracy'])
print 'Model compield in {0} seconds'.format(time.time() - start_time)

Model compield in 0.101611852646 seconds


In [120]:
model.fit(x_train, y_train, epochs=150, batch_size=1)
# model.fit(x_test, y_test, epochs=150,batch_size=20)

Train on 426 samples, validate on 211 samples
Epoch 1/150
426/426 [==============================] - 0s - loss: 551995691532891712.0000 - acc: 0.1831 - val_loss: 72.7915 - val_acc: 0.0190
Epoch 2/150
426/426 [==============================] - 0s - loss: 60.5258 - acc: 0.1995 - val_loss: 72.6919 - val_acc: 0.0190
Epoch 3/150
426/426 [==============================] - 0s - loss: 60.5681 - acc: 0.1995 - val_loss: 72.7915 - val_acc: 0.0190
Epoch 4/150
426/426 [==============================] - 0s - loss: 60.6362 - acc: 0.1808 - val_loss: 72.6919 - val_acc: 0.0190
Epoch 5/150
426/426 [==============================] - 0s - loss: 60.6056 - acc: 0.1808 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 6/150
426/426 [==============================] - 0s - loss: 60.5751 - acc: 0.1972 - val_loss: 73.0948 - val_acc: 0.0047
Epoch 7/150
426/426 [==============================] - 0s - loss: 60.6268 - acc: 0.1784 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 8/150
426/426 [==============================] - 0

426/426 [==============================] - 0s - loss: 60.5986 - acc: 0.1808 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 65/150
426/426 [==============================] - 0s - loss: 60.6291 - acc: 0.1831 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 66/150
426/426 [==============================] - 0s - loss: 60.5775 - acc: 0.1925 - val_loss: 72.7915 - val_acc: 0.0190
Epoch 67/150
426/426 [==============================] - 0s - loss: 60.6385 - acc: 0.1690 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 68/150
426/426 [==============================] - 0s - loss: 60.5751 - acc: 0.1878 - val_loss: 73.4455 - val_acc: 0.0190
Epoch 69/150
426/426 [==============================] - 0s - loss: 60.6174 - acc: 0.1784 - val_loss: 72.7915 - val_acc: 0.0190
Epoch 70/150
426/426 [==============================] - 0s - loss: 60.6291 - acc: 0.1761 - val_loss: 72.6919 - val_acc: 0.0190
Epoch 71/150
426/426 [==============================] - 0s - loss: 60.5892 - acc: 0.1901 - val_loss: 72.7915 - val_acc: 0.01

426/426 [==============================] - 0s - loss: 60.5446 - acc: 0.2066 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 129/150
426/426 [==============================] - 0s - loss: 60.6033 - acc: 0.1784 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 130/150
426/426 [==============================] - 0s - loss: 60.5845 - acc: 0.1878 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 131/150
426/426 [==============================] - 0s - loss: 60.6268 - acc: 0.1878 - val_loss: 72.6303 - val_acc: 0.5308
Epoch 132/150
426/426 [==============================] - 0s - loss: 60.5352 - acc: 0.2019 - val_loss: 72.7915 - val_acc: 0.0190
Epoch 133/150
426/426 [==============================] - 0s - loss: 60.4765 - acc: 0.1995 - val_loss: 74.6635 - val_acc: 0.0047
Epoch 134/150
426/426 [==============================] - 0s - loss: 60.7207 - acc: 0.1737 - val_loss: 72.6919 - val_acc: 0.0190
Epoch 135/150
426/426 [==============================] - 0s - loss: 60.6127 - acc: 0.1854 - val_loss: 72.7915 - val_ac

In [117]:
# show the accuracy on the testing set
print("[INFO] evaluating on testing set...")
(loss, accuracy) = model.evaluate(x_test, y_test,
	batch_size=128, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,
	accuracy * 100))

[INFO] evaluating on testing set...
128/191 [===================>..........] - ETA: 0s[INFO] loss=64.0105, accuracy: 1.5707%


In [24]:
print y_test

[[  9]
 [  0]
 [203]
 [  0]
 [  0]
 [  0]
 [  0]
 [415]
 [136]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [515]
 [324]
 [189]
 [  1]
 [  0]
 [282]
 [  0]
 [  0]
 [ 35]
 [  0]
 [108]
 [  0]
 [  1]
 [ 18]
 [ 25]
 [  0]
 [203]
 [  0]
 [124]
 [  0]
 [ 87]
 [297]
 [ 11]
 [  0]
 [ 25]
 [136]
 [  0]
 [  0]
 [246]
 [  3]
 [ 79]
 [  0]
 [  0]
 [  0]
 [  0]
 [449]
 [  0]
 [  0]
 [ 58]
 [  0]
 [ 10]
 [ 13]
 [  0]
 [  0]
 [  0]
 [  0]
 [ 87]
 [132]
 [  0]
 [  0]
 [  0]
 [  0]
 [176]
 [ 29]
 [  0]
 [ 63]
 [ 75]
 [  0]
 [  0]
 [  0]
 [119]
 [  0]
 [  0]
 [157]
 [  0]
 [  0]
 [ 81]
 [283]
 [ 67]
 [ 61]
 [154]
 [  0]
 [481]
 [  0]
 [  0]
 [  0]
 [  7]
 [  0]
 [  0]
 [  0]
 [ 72]
 [102]
 [110]
 [  0]
 [  0]
 [  0]
 [  1]
 [  0]
 [  0]
 [  0]
 [  5]
 [316]
 [  0]
 [  2]
 [ 54]
 [  0]
 [297]
 [217]
 [164]
 [284]
 [246]
 [ 28]
 [344]
 [ 40]
 [  0]
 [319]
 [286]
 [ 72]
 [ 16]
 [113]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [ 55]
 [  0]
 [106]
 [  0]
 [132]
 [  0]
 [  0]
 [ 25]
 [  0]